generate choicefile & qivarfiles

In [1]:
# quantize
# copy from makepathweightfiles/makepathweightfiles.ipynb
import random
random.seed(0)

def quantize_to_multiple_of_1_64(x,factor):
    return round(x * factor) / factor

def quantize_and_adjust_with_indices(indexed_values,factor,isprint=False):
    if isprint: print(indexed_values)
    # Quantize each value
    quantized_values = [(i, quantize_to_multiple_of_1_64(v,factor)) for i, v in indexed_values]
    if isprint: print(quantized_values)
    
    # Calculate the current sum
    current_sum = sum(v for _, v in quantized_values)
    
    # Calculate the difference between the sum and 1
    difference = current_sum - 1
    if isprint: print(f"currsum={current_sum}, difference={difference}")

    random_values = [(v, random.random()) for v in quantized_values]
    if isprint: print(random_values)
    
    if difference != 0:
        if difference > 0:
            # Sort indices by quantized values to adjust the largest values first
            sorted_random = sorted(random_values, key=lambda x: (x[0][1],x[1]), reverse=True)
            # sorted_quantized = sorted(quantized_values, key=lambda x: x[1], reverse=True)
            sorted_quantized = [v for v,_ in sorted_random]
            if isprint: print(sorted_random)
            if isprint: print(sorted_quantized)
            
            while difference !=0:
                for i, value in sorted_quantized:
                    if difference == 0:
                        break
                    # Calculate adjustment
                    # adjustment = min(difference, value % (1 / factor))
                    # adjustment = difference
                    new_value = value - 1.0/factor
                    
                    # Update the quantized values
                    quantized_values = [(idx, new_value if idx == i else v) for idx, v in quantized_values]
                    difference -= 1.0/factor
                
        else:
            # If the difference is negative (i.e., sum is less than 1), we need to increase values
            # sorted_quantized = sorted(quantized_values, key=lambda x: x[1])
            sorted_random = sorted(random_values, key=lambda x: (x[0][1],x[1]))
            # sorted_quantized = sorted(quantized_values, key=lambda x: x[1], reverse=True)
            sorted_quantized = [v for v,_ in sorted_random]
            
            while difference != 0:
                for i, value in sorted_quantized:
                    if difference == 0:
                        break
                    # Calculate adjustment
                    # adjustment = min(-difference, (1 / factor) - (value % (1 / factor)))
                    # adjustment = -difference
                    new_value = value + 1.0/factor
                    
                    # Update the quantized values
                    quantized_values = [(idx, new_value if idx == i else v) for idx, v in quantized_values]
                    difference += 1.0/factor
    
    # Return the list sorted by original indices
    # return [v for i, v in sorted(quantized_values)]
    if isprint: print(quantized_values)
    return quantized_values

def check(qvarfile, numsw):
    # check qvarfile
    checkweightarr = list()
    for i in range(numsw):
        checkweightarr.append(list())
        for j in range(numsw):
            checkweightarr[i].append(0)
    with open(qvarfile,'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.split(',')
            fromsw = int(tokens[0])
            tosw = int(tokens[1])
            pid = int(tokens[2])
            weight = float(tokens[3])
            checkweightarr[fromsw][tosw] += weight

    for fromsw in range(numsw):
        for tosw in range(numsw):
            if checkweightarr[fromsw][tosw]!=0 and checkweightarr[fromsw][tosw]!=1:
                print(f"from {fromsw} to {tosw}: weight={checkweightarr[fromsw][tosw]}")

def quantize(varfile, qvarfile, factor, numsw):
    with open(varfile,'r') as f:
        lines = f.readlines()[1:]

    valuearr = list()
    weightarr = list()
    qweightarr = list()
    for i in range(numsw):
        valuearr.append(list())
        weightarr.append(list())
        qweightarr.append(list())
        for j in range(numsw):
            valuearr[i].append(list())
            weightarr[i].append(list())
            qweightarr[i].append(list()) 

    for line in lines:
        tokens = line.split(',')
        varname = tokens[0]
        vartokens = varname.split('_')
        fromsw = int(vartokens[0])
        tosw = int(vartokens[1])
        pid = int(vartokens[2])
        value = float(tokens[1])
        valuearr[fromsw][tosw].append([pid,value])
        weightarr[fromsw][tosw].append([pid,value])

    for fromsw in range(numsw):
        for tosw in range(numsw):
            if len(valuearr[fromsw][tosw])>0:
                valuesum = 0
                for _,value in valuearr[fromsw][tosw]:
                    valuesum += value
                valuetoweight = 1.0/valuesum
                for ipv,pv in enumerate(weightarr[fromsw][tosw]):
                    weightarr[fromsw][tosw][ipv][1] = pv[1] * valuetoweight

    for fromsw in range(numsw):
        for tosw in range(numsw):
            if len(weightarr[fromsw][tosw])>0:
                qweightarr[fromsw][tosw] = quantize_and_adjust_with_indices(weightarr[fromsw][tosw],factor)

    with open(qvarfile,'w') as f:
        for fromsw in range(numsw):
            for tosw in range(numsw):
                if len(qweightarr[fromsw][tosw])>0:
                    for pid,weight in qweightarr[fromsw][tosw]:
                        if weight>0:
                            f.write(f"{fromsw},{tosw},{pid},{weight}\n")

    check(qvarfile, numsw)

In [5]:
import shutil

# cluster_c
H=2
factor=64
numsw=80
resultfile = f"/home/annzhou/DRing/src/emp/datacentre/computerouting2/resultfiles/result_dring_0_0_cluster_c_2_0"
choicedict = dict()
for i in range(24,48):
    choicedict[i] = 0
with open(resultfile,'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split(',')
        myH = int(tokens[0])
        if myH == H:
            solveinterval = int(tokens[1])
            # routing = tokens[3]
            computeinterval = int(tokens[4])
            choicedict[solveinterval] = computeinterval

for interval in range(24,48):
    myfile = f"/home/annzhou/DRing/src/emp/datacentre/qivarfiles/qivar_dring_0_0_cluster_c_{interval}_su3_2_0"
    if interval in range(30,42):
        # just copy
        copyfromfile = f"/home/annzhou/DRing/src/emp/datacentre/qivarfiles/routing_cluster_c/qivar_dring_0_0_cluster_c_{interval}_su3_2_0"
        shutil.copy(copyfromfile, myfile)
    else:
        # need to quantize
        ivarfile = f"/home/annzhou/DRing/src/emp/datacentre/qivarfiles/routing_cluster_c/ivar_dring_0_0_su3_cluster_c_{choicedict[interval]}_2_0"
        quantize(ivarfile, myfile, factor, numsw)

generate conffiles

In [6]:
trafficname = "cluster_c"
multstrarr1 = ["0 19 100","0 38 100","0 57 100","0 75 100","0 94 100","1 13 100","1 32 100","1 51 100","1 70 100"]
multnamearr1 = range(10,91,10)
sseedarr = range(1,6)

conffile = f"/home/annzhou/DRing/src/emp/datacentre/experiments/routing2/{trafficname}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 24 48 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 24 48 0 0 0 0 null 0 {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_{trafficname}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"